<a href="https://colab.research.google.com/github/seuhihihi/Colab-Python/blob/main/%E5%B7%B2%E6%9C%89%E6%95%B0%E6%8D%AE%E4%B8%BB%E6%88%90%E5%88%86%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#数据格式示例
#ExperimentID	Temperature	FlowRate	SampleSize	ParticleSize	Olefins	Paraffins	Diolefins	Aromatics
#1	480	100	50	100	12.3	5.1	2.3	0.20
#2	540	200	100	200	18.2	5.9	3.1	0.32

import pandas as pd

# 读取 Excel 或 CSV 文件（请将文件名替换为你的数据文件）
df = pd.read_excel('your_data.xlsx')  # 或 pd.read_csv('your_data.csv')

# 显示前几行确认结构

print(df.head())
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 示例：对“烯烃”产率进行 ANOVA 分析
model = ols('Olefins ~ C(Temperature) + C(FlowRate) + C(SampleSize) + C(ParticleSize)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# 选择你想用于 PCA 的产物列（可以有 4–10 个甚至更多）
features = ['Olefins', 'Paraffins', 'Diolefins', 'Aromatics']
X = df[features]

# 执行 PCA 并将前两个主成分加入原表
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X)
df['PC1'] = pca_result[:, 0]
df['PC2'] = pca_result[:, 1]

# 可视化实验分布（按温度分组示意）
plt.figure(figsize=(8,6))
sns.scatterplot(data=df, x='PC1', y='PC2', hue='Temperature', palette='coolwarm', s=80)
plt.title('PCA 主成分分布图 - 实验产物')
plt.grid(True)
plt.tight_layout()
plt.show()

# 假设你有一组 "正常BBD条件" 数据 和 一组 "极端条件" 数据
# 可以通过条件筛选获得均值，例如：
bbd = df[df['SampleSize'] <= 150].mean(numeric_only=True)
extreme = df[df['SampleSize'] >= 1000].mean(numeric_only=True)

# 选择几个关键产物类别进行比较
classes = ['Olefins', 'Paraffins', 'Diolefins', 'Aromatics']
rel_change = ((extreme[classes] - bbd[classes]) / bbd[classes]) * 100

# 可视化结果
rel_change.sort_values().plot(kind='barh', color='orange', figsize=(6,4))
plt.xlabel('Relative Change (%)')
plt.title('极端条件 vs BBD 平均产率变化')
plt.grid(axis='x')
plt.tight_layout()
plt.show()
